In [33]:
from __future__ import print_function
from __future__ import division
import os
import os.path as op
from itertools import groupby
from collections import defaultdict
import subprocess
import pandas as pd

In [3]:
with open("phage_by_group.txt") as prelim_groups:
    phage_grp = {}
    phages = []
    groups = []
    for line in prelim_groups:
        line = line.rstrip("\n")
        vec = line.split("\t")
        phage_grp[vec[0]] = vec[1]
        phages.append(vec[0])
        groups
    groups = list(phage_grp.values())
    phages = list(phage_grp.keys())

phagepre = []
phage_reps = []
phage_rep_suffix = defaultdict(lambda: [])


for p in phages:
    pre = ".".join(p.split(".")[0:2])
    if pre not in phagepre:
        phagepre.append(pre)
    else:
        phage_reps.append(pre)
        
repgroups = []

for r in phage_reps:
    for g in phage_grp.keys():
        if r in g:
            repgroups.append(phage_grp[g].split(":")[-1])
            break

for p in phages:
    pre = ".".join(p.split(".")[0:2])
    suffix = p.split(".")[-2]
    if pre in phage_reps:
        phage_rep_suffix[pre].append(suffix)
print(phage_rep_suffix)

defaultdict(<function <lambda> at 0x103ed0a28>, {'1.247': ['B', 'A'], '2.159': ['B', 'A'], '1.233': ['B', 'A'], '1.283': ['A', 'C', 'B'], '1.118': ['B', 'A'], '1.268': ['A', 'B'], '1.115': ['B', 'A'], '1.263': ['B', 'A'], '1.139': ['B', 'A'], '2.095': ['B', 'A'], '1.111': ['A', 'B'], '1.021': ['A', 'B', 'C'], '1.189': ['B', 'O', 'C'], '1.188': ['A', 'B', 'C'], '1.034': ['O', 'X'], '1.237': ['B', 'A'], '1.217': ['O', 'Y'], '1.238': ['A', 'B'], '1.215': ['B', 'A'], '1.214': ['O', 'Y'], '1.211': ['A', 'B'], '1.249': ['B', 'A'], '1.107': ['B', 'C', 'A'], '1.122': ['A', 'B'], '1.277': ['A', 'B'], '1.271': ['B', 'A'], '1.270': ['B', 'A'], '1.198': ['B', 'A'], '1.199': ['B', 'A']})


In [68]:
def get_position(pos):
    if pos == "NA":
        return 
    else:
        return int(pos)

class Replicates():
    def __init__(self, name, query, sub):
        self.name = name
        self.query = query
        self.sub = sub
        self.bqfasta = "./step1contig1/{name}.{query}.step1.contig1.fasta".format(**locals())
        self.bsfasta = "./step1contig1/{name}.{sub}.step1.contig1.fasta".format(**locals())
        self.fqfasta = "./final_fastas_bp/{name}.{query}.final.fasta".format(**locals())
        self.fsfasta = "./final_fastas_bp/{name}.{sub}.final.fasta".format(**locals())
        self.mafbefore = "./last_aln/{name}_before.maf".format(name=name)
        self.mafdotbefore = "./last_aln/{name}_{query}{sub}_before_dottbl.txt".format(**locals())
        self.mafafter = "./last_aln/{name}_after.maf".format(name=name)
        self.mafdotafter = "./last_aln/{name}_{query}{sub}_after_dottbl.txt".format(**locals())
    
    def run_last(self):
        cmd1 = "lastdb {self.name}_before {self.bqfasta}".format(**locals())
        cmd2 = "lastal {self.name}_before {self.bsfasta} > {self.mafbefore}".format(**locals())
        cmd3 = "lastdb {self.name}_after {self.fqfasta}".format(**locals())
        cmd4 = "lastal {self.name}_after {self.fsfasta} > {self.mafafter}".format(**locals())
        subprocess.call(cmd1, shell=True)
        subprocess.call(cmd2, shell=True)
        subprocess.call(cmd3, shell=True)
        subprocess.call(cmd4, shell=True)
        print("last successfully run for {self.name}".format(**locals()))
        maf_to_tbl(self.mafbefore, self.mafdotbefore, self.query, self.sub, self.bqfasta, self.bsfasta)
        maf_to_tbl(self.mafafter, self.mafdotafter, self.query, self.sub, self.fqfasta, self.fsfasta)
        print("maf tables converted to list for {self.name}".format(**locals()))

    def get_comp_ani(self):
        if op.exists(self.mafdotafter) == False:
            self.run_last()    
        with open(self.mafdotafter) as ih:
            qmatch = set()
            qmax = 0
            smatch = set()
            smax = 0
            for i, l in enumerate(ih):
                if i > 0:
                    q = get_position(l.split("\t")[0])
                    s = get_position(l.split("\t")[1])
                    m = int(l.split("\t")[2])
                    if m == 1 and q > 0 and s > 0:
                        qmatch.add(q)
                        smatch.add(s)
                    if qmax < q:
                        qmax = q
                    if smax < s:
                        smax = s
            self.ani = ((len(qmatch)+len(smatch))/2)/((smax+qmax)/2)
            self.smm = smax - len(smatch)
            self.qmm = qmax - len(qmatch)
            
# python functions to create a synteny table from maf output:
def read_fasta(file_handle):
    '''Fasta iterator'''
    for header, group in groupby(file_handle, lambda line: line[0] == '>'):
        if header:
            line = next(group)
            name = line[1:].strip()
        else:
            seq = ''.join(line.strip() for line in group)
            yield name, seq

def get_fasta_len(fasta):
    genome_len = 0
    with open(fasta) as ih:
        for name, seq in read_fasta(ih):
            genome_len += len(seq)
    return genome_len

def real_start(start, glen, strand):
    if strand == "+":
        rs = start + 1
    if strand == "-":
        rs = glen - (start+1)
    return rs

def advance_seq(location, strand):
    if strand == "+":
        location += 1
    elif strand == "-":
        location -= 1
    else:
        print("strand not designated!")
        raise IOError
    return location

def maf_to_tbl(maf, outfile, query, subject, query_fasta, subject_fasta):
    with open(outfile, "w") as oh:
        print(query, subject, "match", sep="\t", file=oh)
        ih = [i for i in open(maf).readlines() if i.startswith("#")==False and len(i.split())>1]
        qlen = get_fasta_len(query_fasta)
        slen = get_fasta_len(subject_fasta)
        for i in range(0, len(ih), 3):
            scores = ih[i]
            aln1 = ih[i+1]
            #print(aln1.split()[0:3])
            aln2 = ih[i+2]
            #print(aln2.split()[0:3])

            strand1 = aln1.split()[4]
            start1 = real_start(int(aln1.split()[2]), qlen, strand1)
            seq1 = aln1.split()[-1]

            strand2 = aln2.split()[4]
            start2 = real_start(int(aln2.split()[2]), slen, strand2)
            seq2 = aln2.split()[-1]
            for i, (n1, n2) in enumerate(zip(seq1, seq2)):
                start1 = advance_seq(start1, strand1)
                start2 = advance_seq(start2, strand2)
                print(start1, start2, compare_seqs(n1,n2), sep="\t", file=oh)
        for i in range(1, qlen+1):
            print(i,"NA", 0, sep="\t", file=oh)
        for i in range(1, slen+1):
            print("NA",i,0,sep="\t", file=oh)
    return outfile

def compare_seqs(n1, n2):
    if n1 == n2:
        return 1
    else:
        return 0

In [69]:
with open("./last_aln/rep_comparison_tbl.txt", "w") as oh:
    print("name", "query", "sub", "query_mismatch", "sub_mismatch", "ani", sep="\t", file=oh)
    for p in phage_rep_suffix:
        reps = phage_rep_suffix[p]
        if len(reps) == 2 and "Y" in reps or "X" in reps:
            continue
        elif len(reps) == 3 and "Y" not in reps and "X" not in reps:
            r = Replicates(p, reps[0], reps[1])
            r.get_comp_ani()
            print(r.name, r.query, r.sub, r.qmm, r.smm, r.ani, sep="\t", file=oh)
            r1 = Replicates(p, reps[0], reps[2])
            r1.get_comp_ani()
            print(r1.name, r1.query, r1.sub, r1.qmm, r1.smm, r1.ani, sep="\t", file=oh)
            r2 = Replicates(p, reps[1], reps[2])
            r2.get_comp_ani()
            print(r2.name, r2.query, r2.sub, r2.qmm, r2.smm, r2.ani, sep="\t", file=oh)
        else:
            r = Replicates(p, reps[0], reps[1])
            r.get_comp_ani()
            print(r.name, r.query, r.sub, r.qmm, r.smm, r.ani, sep="\t", file=oh)

last successfully run for 1.247
maf tables converted to list for 1.247
last successfully run for 2.159
maf tables converted to list for 2.159
last successfully run for 1.233
maf tables converted to list for 1.233
last successfully run for 1.283
maf tables converted to list for 1.283
last successfully run for 1.283
maf tables converted to list for 1.283
last successfully run for 1.283
maf tables converted to list for 1.283
last successfully run for 1.118
maf tables converted to list for 1.118
last successfully run for 1.268
maf tables converted to list for 1.268
last successfully run for 1.115
maf tables converted to list for 1.115
last successfully run for 1.263
maf tables converted to list for 1.263
last successfully run for 1.139
maf tables converted to list for 1.139
last successfully run for 2.095
maf tables converted to list for 2.095
last successfully run for 1.111
maf tables converted to list for 1.111
last successfully run for 1.021
maf tables converted to list for 1.021
last s

In [55]:
r = Replicates("1.249","B", "A")

In [66]:
with open(r.mafdotafter) as ih:
    qmatch = set()
    qmax = 0
    smatch = set()
    smax = 0
    for i, l in enumerate(ih):
        if i > 0:
            q = get_position(l.split("\t")[0])
            s = get_position(l.split("\t")[1])
            m = int(l.split("\t")[2])
            if m == 1 and q > 0 and s > 0:
                qmatch.add(q)
                smatch.add(s)
            if qmax < q:
                qmax = q
            if smax < s:
                smax = s
    r.ani = ((len(qmatch)+len(smatch))/2)/((smax+qmax)/2)
    r.smm = smax - len(smatch)
    r.qmm = qmax - len(qmatch)

In [67]:
for s in smatch:
    if s > 0 and s < smax:
        continue
    else:
        print(s)

10612
